# Cohesive Weather Forecasting & Satellite Analysis Framework

This notebook implements a comprehensive framework for processing satellite imagery for various weather forecasting tasks. It includes:

1.  **Unified Configuration**: Centralized management of hyperparameters and paths.
2.  **Data Pipeline**: Robust loading for the existing GIBS dataset (used for TMAX prediction).
3.  **Model Zoo**: Implementation of 6 specific deep learning architectures from academic literature:
    *   **GIBSForecaster**: The existing CNN+GRU model for TMAX prediction.
    *   **CloudCoverNowcaster**: CNN for cloud cover segmentation (Berthomier et al., 2020).
    *   **SIANet**: 3D-CNN for spatiotemporal rainfall prediction (Seo et al., 2022).
    *   **Pix2PixLST**: cGAN for Land Surface Temperature & Emissivity (Garg et al., 2023).
    *   **MicrowaveLSTNet**: CNN for LST from microwave data (Wang et al., 2020).
    *   **ConvectionCNN**: CNN for intense convection detection (Cintineo et al., 2020).
4.  **Training & Verification**: A unified training loop for the active task and verification steps for all model architectures.

In [ ]:
import os
import glob
import random
from datetime import datetime, timedelta
from pathlib import Path
from collections import defaultdict
import math

import pandas as pd
import numpy as np
from PIL import Image, UnidentifiedImageError
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as T

# Ensure reproducibility
def set_seed(seed=42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)

set_seed(42)

print(f"PyTorch: {torch.__version__}")
device = torch.device("cuda" if torch.cuda.is_available() else ("mps" if torch.backends.mps.is_available() else "cpu"))
print(f"Using device: {device}")

In [ ]:
class Config:
    # Paths
    DATA_ROOT = Path("/content/drive/MyDrive/DL/")
    GIBS_ROOT = DATA_ROOT / "gibs"
    DAILIES_PATH = DATA_ROOT / "dailies.csv"
    PROCESSED_DIR = DATA_ROOT / "processed_tensors"
    
    # Data Parameters
    LOCATION = "new_york_ny"
    HISTORY_DAYS = 30
    IMAGE_SIZE = (128, 128)
    TARGET_COLUMN = "target_tmax_next_day"
    
    # Training Hyperparameters
    BATCH_SIZE = 32
    LEARNING_RATE = 1e-4
    NUM_EPOCHS = 10
    
    # Model Specific Defaults
    # Number of channels will be inferred from data for the main task
    # but we set defaults for others
    DEFAULT_CHANNELS = 3
    
config = Config()
config.PROCESSED_DIR.mkdir(exist_ok=True, parents=True)

## Data Loading (GIBS Dataset)
This section handles the discovery and loading of the GIBS satellite imagery for the primary TMAX forecasting task.

In [ ]:
class GIBSSequenceDataset(Dataset):
    def __init__(self, end_dates, processed_dir, target_df, history_days=30, num_channels_expected=162):
        self.end_dates = end_dates
        self.processed_dir = processed_dir
        self.target_df = target_df
        self.history_days = history_days
        self.num_channels_expected = num_channels_expected

    def __len__(self):
        return len(self.end_dates)

    def __getitem__(self, idx):
        end_date = self.end_dates[idx]
        # Calculate the 30-day window: [end_date - 29 days, ..., end_date]
        window = [end_date - timedelta(days=i) for i in range(self.history_days - 1, -1, -1)]

        frames = []
        for d in window:
            tensor_path = self.processed_dir / f"{d}.pt"
            if tensor_path.exists():
                # Load (C, H, W)
                frames.append(torch.load(tensor_path, weights_only=True))
            else:
                # Fallback: create zeros
                frames.append(torch.zeros((self.num_channels_expected, config.IMAGE_SIZE[0], config.IMAGE_SIZE[1])))

        # Stack frames -> (History, Channels, H, W)
        x = torch.stack(frames, dim=0)

        # Get Target
        try:
            val = self.target_df.loc[pd.Timestamp(end_date), config.TARGET_COLUMN]
            y = torch.tensor(val, dtype=torch.float32)
        except KeyError:
            y = torch.tensor(0.0, dtype=torch.float32)

        return x, y

def prepare_data(config):
    # 1. Load Targets
    if not config.DAILIES_PATH.exists():
        print("Dailies CSV not found. Running in Demo Mode with dummy data.")
        return None, None, 162
        
    df = pd.read_csv(config.DAILIES_PATH)
    df["DATE"] = pd.to_datetime(df["DATE"])
    df = df.sort_values("DATE").set_index("DATE")
    target = df[["TMAX"]].rename(columns={"TMAX": config.TARGET_COLUMN})
    target = target.shift(-1)  # Predict next day
    target = target.dropna()
    
    # 2. Scan Processed Tensors
    processed_files = sorted(list(config.PROCESSED_DIR.glob("*.pt")))
    if not processed_files:
        print("No processed tensors found. Please run the preprocessing script first or ensure data is mounted.")
        return None, None, 162
    
    # Infer channels
    sample = torch.load(processed_files[0], map_location="cpu")
    num_channels = sample.shape[0]
    
    processed_dates = set()
    for pf in processed_files:
        try:
            processed_dates.add(datetime.strptime(pf.stem, "%Y-%m-%d").date())
        except ValueError: pass
        
    # 3. Find Valid Sequences
    valid_dates = []
    target_dates = set(target.index.date)
    sorted_dates = sorted(list(processed_dates))
    available_set = set(sorted_dates)
    
    for d in sorted_dates:
        if d not in target_dates:
            continue
        # Check contiguous window
        is_valid = True
        for i in range(config.HISTORY_DAYS):
            if (d - timedelta(days=i)) not in available_set:
                is_valid = False
                break
        if is_valid:
            valid_dates.append(d)
            
    # 4. Split
    split_idx = int(len(valid_dates) * 0.8)
    train_dates = valid_dates[:split_idx]
    test_dates = valid_dates[split_idx:]
    
    train_ds = GIBSSequenceDataset(train_dates, config.PROCESSED_DIR, target, config.HISTORY_DAYS, num_channels)
    test_ds = GIBSSequenceDataset(test_dates, config.PROCESSED_DIR, target, config.HISTORY_DAYS, num_channels)
    
    return train_ds, test_ds, num_channels

# Model Zoo
Here we implement the 6 requested models.

In [ ]:
class DayEncoder(nn.Module):
    """Used by GIBSForecaster to encode one day's multi-channel image."""
    def __init__(self, in_channels, hidden_dim=128):
        super().__init__()
        self.net = nn.Sequential(
            nn.Conv2d(in_channels, 32, kernel_size=3, padding=1), nn.ReLU(), nn.MaxPool2d(2),
            nn.Conv2d(32, 64, kernel_size=3, padding=1), nn.ReLU(), nn.MaxPool2d(2),
            nn.Conv2d(64, 128, kernel_size=3, padding=1), nn.ReLU(), nn.AdaptiveAvgPool2d((1,1)),
        )
        self.proj = nn.Linear(128, hidden_dim)

    def forward(self, x):
        b, c, h, w = x.shape
        z = self.net(x)
        z = z.view(b, -1)
        return self.proj(z)

class GIBSForecaster(nn.Module):
    """
    Current implementation for TMAX forecasting.
    Input: (B, History, C, H, W)
    Output: (B) -> Scalar Tmax
    """
    def __init__(self, in_channels, history_days, hidden_dim=128, rnn_hidden=256):
        super().__init__()
        self.day_encoder = DayEncoder(in_channels, hidden_dim)
        self.rnn = nn.GRU(hidden_dim, rnn_hidden, batch_first=True)
        self.head = nn.Sequential(nn.Linear(rnn_hidden, 128), nn.ReLU(), nn.Linear(128, 1))

    def forward(self, x):
        b, t, c, h, w = x.shape
        x = x.view(b * t, c, h, w)
        enc = self.day_encoder(x)  # (b*t, hidden)
        enc = enc.view(b, t, -1)
        out, _ = self.rnn(enc)
        last = out[:, -1, :]
        return self.head(last).squeeze(1)

In [ ]:
class CloudCoverNowcaster(nn.Module):
    """
    Model 1: 'Cloud Cover Nowcasting' (Berthomier et al., 2020)
    Objective: Predict next 6 timesteps of cloud cover.
    Architecture: CNN (U-Net style or SegNet style).
    Input: (B, 4, H, W) -> 4 consecutive grayscale images (or channels)
    Output: (B, 6, H, W) -> Probabilities for next 6 timesteps
    """
    def __init__(self, in_channels=4, out_timesteps=6):
        super().__init__()
        
        # Encoder
        self.enc1 = nn.Sequential(nn.Conv2d(in_channels, 64, 3, padding=1), nn.BatchNorm2d(64), nn.ReLU())
        self.enc2 = nn.Sequential(nn.Conv2d(64, 128, 3, padding=1), nn.BatchNorm2d(128), nn.ReLU())
        self.enc3 = nn.Sequential(nn.Conv2d(128, 256, 3, padding=1), nn.BatchNorm2d(256), nn.ReLU())
        
        self.pool = nn.MaxPool2d(2, 2)
        
        # Bottleneck
        self.bottleneck = nn.Sequential(nn.Conv2d(256, 512, 3, padding=1), nn.BatchNorm2d(512), nn.ReLU())
        
        # Decoder (Upsampling)
        self.up = nn.Upsample(scale_factor=2, mode='bilinear', align_corners=True)
        
        self.dec3 = nn.Sequential(nn.Conv2d(512 + 256, 256, 3, padding=1), nn.BatchNorm2d(256), nn.ReLU())
        self.dec2 = nn.Sequential(nn.Conv2d(256 + 128, 128, 3, padding=1), nn.BatchNorm2d(128), nn.ReLU())
        self.dec1 = nn.Sequential(nn.Conv2d(128 + 64, 64, 3, padding=1), nn.BatchNorm2d(64), nn.ReLU())
        
        # Final prediction head
        self.head = nn.Conv2d(64, out_timesteps, 1)

    def forward(self, x):
        # x: (B, 4, H, W)
        e1 = self.enc1(x)
        p1 = self.pool(e1)
        
        e2 = self.enc2(p1)
        p2 = self.pool(e2)
        
        e3 = self.enc3(p2)
        p3 = self.pool(e3)
        
        b = self.bottleneck(p3)
        
        # Decode with Skip Connections
        d3 = self.dec3(torch.cat([self.up(b), e3], dim=1))
        d2 = self.dec2(torch.cat([self.up(d3), e2], dim=1))
        d1 = self.dec1(torch.cat([self.up(d2), e1], dim=1))
        
        # Output: (B, 6, H, W)
        return torch.sigmoid(self.head(d1))

In [ ]:
class SIANet(nn.Module):
    """
    Model 2: 'SIANet' (Seo et al., 2022)
    Objective: Spatiotemporal rainfall prediction.
    Architecture: 3D-CNN with Context Aggregation.
    Input: (B, C, Depth/Time=4, H, W)
    Output: (B, 1, Depth/Time=32, H, W)
    """
    def __init__(self, in_channels=4, input_frames=4, output_frames=32):
        super().__init__()
        # 3D Convolution to process spatial+temporal
        self.conv1 = nn.Conv3d(in_channels, 32, kernel_size=(3,3,3), padding=(1,1,1))
        self.bn1 = nn.BatchNorm3d(32)
        
        # Context Aggregation (Dilated or Large Kernel)
        self.context = nn.Conv3d(32, 64, kernel_size=(3,5,5), padding=(1,2,2))
        self.bn2 = nn.BatchNorm3d(64)
        
        # Residual Block Sim
        self.res_conv = nn.Conv3d(64, 64, kernel_size=3, padding=1)
        
        # Output generation: Upsample in time domain if needed 
        # (Simple implementation: Linear projection/Expansion)
        self.temporal_expand = nn.ConvTranspose3d(64, 32, kernel_size=(output_frames//input_frames, 1, 1), stride=(output_frames//input_frames, 1, 1))
        
        self.out = nn.Conv3d(32, 1, kernel_size=1)

    def forward(self, x):
        # x: (B, C, T, H, W)
        x = F.relu(self.bn1(self.conv1(x)))
        x = F.relu(self.bn2(self.context(x)))
        
        # Residual connection
        res = self.res_conv(x)
        x = F.relu(x + res)
        
        # Expand time dimension (Input 4 -> Output 32)
        x = self.temporal_expand(x)
        
        # Output: (B, 1, 32, H, W)
        return self.out(x)

In [ ]:
class Pix2PixGenerator(nn.Module):
    """
    Model 3: Pix2Pix Generator (Garg et al., 2023)
    Objective: Image-to-Image translation (Sat -> LST & Emissivity)
    Input: (B, C, H, W)
    Output: (B, 2, H, W) -> [LST, Emissivity]
    """
    def __init__(self, in_channels=3, out_channels=2):
        super().__init__()
        # Simple U-Net Generator
        self.down1 = nn.Conv2d(in_channels, 64, 4, 2, 1) # 64
        self.down2 = nn.Conv2d(64, 128, 4, 2, 1) # 32
        self.down3 = nn.Conv2d(128, 256, 4, 2, 1) # 16
        
        self.bottleneck = nn.Conv2d(256, 256, 4, 2, 1) # 8
        
        self.up1 = nn.ConvTranspose2d(256, 256, 4, 2, 1)
        self.up2 = nn.ConvTranspose2d(256+256, 128, 4, 2, 1)
        self.up3 = nn.ConvTranspose2d(128+128, 64, 4, 2, 1)
        self.final = nn.ConvTranspose2d(64+64, out_channels, 4, 2, 1)

    def forward(self, x):
        d1 = F.leaky_relu(self.down1(x), 0.2)
        d2 = F.leaky_relu(self.down2(d1), 0.2)
        d3 = F.leaky_relu(self.down3(d2), 0.2)
        
        b = F.relu(self.bottleneck(d3))
        
        u1 = F.relu(self.up1(b))
        # Skip connections assumed concatenated
        u1 = torch.cat([u1, d3], dim=1)
        
        u2 = F.relu(self.up2(u1))
        u2 = torch.cat([u2, d2], dim=1)
        
        u3 = F.relu(self.up3(u2))
        u3 = torch.cat([u3, d1], dim=1)
        
        return torch.tanh(self.final(u3))

class Pix2PixDiscriminator(nn.Module):
    """
    PatchGAN Discriminator
    Input: (B, C_in + C_out, H, W)
    Output: (B, 1, H/N, W/N)
    """
    def __init__(self, in_channels=3+2):
        super().__init__()
        self.net = nn.Sequential(
            nn.Conv2d(in_channels, 64, 4, 2, 1), nn.LeakyReLU(0.2),
            nn.Conv2d(64, 128, 4, 2, 1), nn.BatchNorm2d(128), nn.LeakyReLU(0.2),
            nn.Conv2d(128, 1, 4, 1, 1) # Output patch predictions
        )
        
    def forward(self, x):
        return self.net(x)

In [ ]:
class MicrowaveLSTNet(nn.Module):
    """
    Model 4: Microwave LST (Wang et al., 2020)
    Objective: Estimate LST from Passive Microwave data.
    Architecture: CNN + FC.
    Input: (B, C, H, W) where C = BTs + Ancillary
    Output: (B, 1, H, W) or (B, 1) depending on resolution. Assuming map output.
    """
    def __init__(self, in_channels=10):
        super().__init__()
        self.features = nn.Sequential(
            nn.Conv2d(in_channels, 32, 3, 1, 1), nn.ReLU(),
            nn.Conv2d(32, 64, 3, 1, 1), nn.ReLU(),
            nn.Conv2d(64, 32, 3, 1, 1), nn.ReLU(),
        )
        self.regressor = nn.Conv2d(32, 1, 1)

    def forward(self, x):
        x = self.features(x)
        return self.regressor(x)

In [ ]:
class ConvectionCNN(nn.Module):
    """
    Model 5: Intense Convection Detection (Cintineo et al., 2020)
    Objective: Binary classification of intense convection.
    Input: (B, C, H, W) -> C = Reflectance, Temp, GLM
    Output: (B, 1) -> Probability
    """
    def __init__(self, in_channels=3):
        super().__init__()
        self.net = nn.Sequential(
            nn.Conv2d(in_channels, 32, 3, 1), nn.ReLU(), nn.MaxPool2d(2),
            nn.Conv2d(32, 64, 3, 1), nn.ReLU(), nn.MaxPool2d(2),
            nn.Conv2d(64, 128, 3, 1), nn.ReLU(), nn.AdaptiveAvgPool2d((1,1)),
            nn.Flatten(),
            nn.Linear(128, 64), nn.ReLU(),
            nn.Linear(64, 1)
        )

    def forward(self, x):
        return torch.sigmoid(self.net(x))

## Training Framework
Unified trainer class for handling different model types.

In [ ]:
class UnifiedTrainer:
    def __init__(self, model, device, optimizer, criterion, task_type="regression"):
        self.model = model
        self.device = device
        self.optimizer = optimizer
        self.criterion = criterion
        self.task_type = task_type

    def train_epoch(self, loader):
        self.model.train()
        total_loss = 0
        for x, y in loader:
            x = x.to(self.device)
            y = y.to(self.device)
            
            self.optimizer.zero_grad()
            
            # Forward pass handling for different input types could go here
            # For now we assume standard x -> y mapping
            pred = self.model(x)
            
            loss = self.criterion(pred, y)
            loss.backward()
            self.optimizer.step()
            
            total_loss += loss.item()
            
        return total_loss / len(loader)

    @torch.no_grad()
    def validate(self, loader):
        self.model.eval()
        total_loss = 0
        for x, y in loader:
            x = x.to(self.device)
            y = y.to(self.device)
            pred = self.model(x)
            loss = self.criterion(pred, y)
            total_loss += loss.item()
        return total_loss / len(loader)

## Execution & Verification

1. **Primary Task**: Train the `GIBSForecaster` on the available TMAX data.
2. **Architecture Verification**: Instantiate all other models with dummy data to verify shape compatibility.

In [ ]:
# 1. Run TMAX Training (Existing Task)
print("=== Starting TMAX Forecast Training ===")
train_loader, test_loader, num_channels = prepare_data(config)

if train_loader:
    train_dl = DataLoader(train_loader, batch_size=config.BATCH_SIZE, shuffle=True)
    test_dl = DataLoader(test_loader, batch_size=config.BATCH_SIZE)
    
    model_tmax = GIBSForecaster(in_channels=num_channels, history_days=config.HISTORY_DAYS).to(device)
    optimizer = torch.optim.Adam(model_tmax.parameters(), lr=config.LEARNING_RATE)
    criterion = nn.L1Loss() # MAE
    
    trainer = UnifiedTrainer(model_tmax, device, optimizer, criterion)
    
    for epoch in range(config.NUM_EPOCHS):
        t_loss = trainer.train_epoch(train_dl)
        v_loss = trainer.validate(test_dl)
        print(f"Epoch {epoch+1}/{config.NUM_EPOCHS} | Train MAE: {t_loss:.4f} | Val MAE: {v_loss:.4f}")
else:
    print("Skipping training run (No data found).")

In [ ]:
# 2. Verify Architectures (Dummy Data)
print("\n=== Verifying New Model Architectures ===")
H, W = 128, 128

def verify_model(name, model, input_shape):
    x = torch.randn(*input_shape).to(device)
    try:
        y = model(x)
        print(f"[PASS] {name}: Input {input_shape} -> Output {y.shape}")
    except Exception as e:
        print(f"[FAIL] {name}: {e}")

# Model 1: Cloud Cover
m1 = CloudCoverNowcaster().to(device)
verify_model("CloudCoverNowcaster", m1, (2, 4, H, W))

# Model 2: SIANet (3D)
m2 = SIANet(in_channels=3, input_frames=4, output_frames=32).to(device)
verify_model("SIANet", m2, (2, 3, 4, H, W))

# Model 3: Pix2Pix Generator
m3_g = Pix2PixGenerator().to(device)
verify_model("Pix2PixGenerator", m3_g, (2, 3, H, W))

# Model 3: Pix2Pix Discriminator
m3_d = Pix2PixDiscriminator().to(device)
# Input to disc is Cat(Real, Fake) -> Channels * 2 + Output channels
verify_model("Pix2PixDiscriminator", m3_d, (2, 5, H, W))

# Model 4: Microwave LST
m4 = MicrowaveLSTNet(in_channels=10).to(device)
verify_model("MicrowaveLSTNet", m4, (2, 10, H, W))

# Model 5: Convection
m5 = ConvectionCNN(in_channels=3).to(device)
verify_model("ConvectionCNN", m5, (2, 3, H, W))

print("\nAll architectures verified.")